In [182]:
import pandas as pd
import numpy as np
import re

In [221]:
data_location = "../Data/properatti_limpio.csv"
data = pd.read_csv(data_location)
data

,Unnamed: 0,property_type,place_name,state_name,description,title,rooms_completo,PILETA,PARRILLA,PATIO_JARDIN,...,surface_total_in_m2_nuevo,surface_covered_in_m2_nuevo,price_usd_per_m2_nuevo,price_per_m2_nuevo,pais,provincia,localidad,zona,barrio,lugar
0,0,PH,Mataderos,Capital Federal,"2 AMBIENTES TIPO CASA PLANTA BAJA POR PASILLO,...",2 AMB TIPO CASA SIN EXPENSAS EN PB,0.0,False,False,False,...,55.0,40.0,1127.272727,19890.163636,Argentina,Capital Federal,Mataderos,Mataderos,Mataderos,Mataderos
1,1,apartment,La Plata,Bs.As. G.B.A. Zona Sur,Venta de departamento en décimo piso al frente...,VENTA Depto 2 dorm. a estrenar 7 e/ 36 y 37 ...,2.0,False,False,False,...,NaN,NaN,NaN,NaN,Argentina,Bs.As. G.B.A. Zona Sur,La Plata,La Plata,La Plata,La Plata
2,2,apartment,Mataderos,Capital Federal,2 AMBIENTES 3ER PISO LATERAL LIVING COMEDOR AM...,2 AMB 3ER PISO CON ASCENSOR APTO CREDITO,0.0,False,False,False,...,55.0,55.0,1309.090909,23098.254545,Argentina,Capital Federal,Mataderos,Mataderos,Mataderos,Mataderos
3,3,PH,Liniers,Capital Federal,PH 3 ambientes con patio. Hay 3 deptos en lote...,PH 3 amb. cfte. reciclado,2.0,False,False,True,...,NaN,NaN,NaN,NaN,Argentina,Capital Federal,Liniers,Liniers,Liniers,Liniers
4,4,apartment,Centro,Buenos Aires Costa Atlántica,DEPARTAMENTO CON FANTÁSTICA ILUMINACIÓN NATURA...,DEPTO 2 AMB AL CONTRAFRENTE ZONA CENTRO/PLAZA ...,1.0,False,False,False,...,35.0,35.0,1828.571429,32264.228571,Argentina,Buenos Aires Costa Atlántica,Mar del Plata,Centro,Centro,Centro
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121215,121215,apartment,Belgrano,Capital Federal,TORRE FORUM ALCORTA - MÁXIMA CATEGORÍA.Impecab...,Torre Forum Alcorta- Impecable 3 ambientes,1.0,True,True,False,...,113.0,93.0,7699.115044,135847.035398,Argentina,Capital Federal,Belgrano,Belgrano,Belgrano,Belgrano
121216,121216,house,Beccar,Bs.As. G.B.A. Zona Norte,Excelente e impecable casa en Venta en Las Lom...,Ruca Inmuebles | Venta | Lomas de San Isidro |...,1.0,True,True,True,...,360.0,360.0,1383.333333,24408.225000,Argentina,Bs.As. G.B.A. Zona Norte,San Isidro,Beccar,Beccar,Beccar
121217,121217,apartment,Villa Urquiza,Capital Federal,VENTA DEPARTAMENTO AMBIENTE DIVISIBLE A ESTREN...,VENTA DEPARTAMENTO AMBIENTE DIVISIBLE A ESTREN...,0.0,False,False,False,...,46.0,39.0,2858.695652,50440.255435,Argentina,Capital Federal,Villa Urquiza,Villa Urquiza,Villa Urquiza,Villa Urquiza
121218,121218,apartment,Plaza Colón,Buenos Aires Costa Atlántica,"2 Amb al contrafrente, luminoso. El departame...",2 amb. C/ dep. de servicio al contrafrente| Re...,0.0,False,False,False,...,48.0,48.0,1997.916667,35252.240625,Argentina,Buenos Aires Costa Atlántica,Mar del Plata,Plaza Colón,Plaza Colón,Plaza Colón


In [192]:
data_m2_vs_description = data.loc[:,["surface_total_in_m2_nuevo","description"]]

In [185]:
mask = data_m2_vs_description["surface_total_in_m2_nuevo"].isnull()
data_m2_vs_description_nulos = data_m2_vs_description[mask]
data_m2_vs_description_nulos

,surface_total_in_m2_nuevo,description
1,NaN,Venta de departamento en décimo piso al frente...
3,NaN,PH 3 ambientes con patio. Hay 3 deptos en lote...
9,NaN,MEDNOZA AL 7600A UNA CUADRA DE CALLE MENDOZAWH...
11,NaN,"Entrada de Coche, Jardin, Living en desnivel, ..."
65,NaN,CODIGO: 1092-VENTA 6886 ubicado en: COMBET 688...
...,...,...
121155,NaN,Casa a reciclar en una planta.130 m2 cubiertos...
121156,NaN,Casa a reciclar en una planta.130 m2 cubiertos...
121207,NaN,Puertos- Barrio Vistas - L. 88Casa en venta so...
121208,NaN,Puertos- Barrio Vistas - L. 88Casa en venta so...


In [193]:
description_serie = data_m2_vs_description_nulos["description"]

In [194]:
patron2 = "\d+( ?)m2(c?)|\d+( ?)m²|\d+( ?)mts|\d+( ?)MT|\d+( ?) metros|\d+( ?)mt2|\d+( ?)M2(C?)"
regex2 = re.compile(patron2)
resultado = description_serie.apply(lambda x:  x if x is np.NaN else regex2.search(x))

In [195]:
numeros_match = resultado[resultado.notnull()].apply(lambda x: x.group(0))

In [196]:
sub_patron = "\d+"
sub_regex = re.compile(sub_patron)
resultado2 = numeros_match.apply(lambda x:  x if x is np.NaN else sub_regex.search(x))

In [198]:
numeros_match2 = resultado2[resultado2.notnull()].apply(lambda x: x.group(0))

In [220]:
surface_total_m2_actualizado = data["surface_total_in_m2_nuevo"].fillna(numeros_match2)
surface_total_m2_actualizado.head()

0     55
1    NaN
2     55
3    NaN
4     35
Name: surface_total_in_m2_nuevo, dtype: object

In [216]:
surface_total_m2_actualizado.isnull().sum()

26623

In [235]:
surface_total_m2_actualizado_con_sup_covered = surface_total_m2_actualizado.fillna(data.loc[:,"surface_covered_in_m2_nuevo"])
surface_total_m2_actualizado_con_sup_covered.isnull().sum()

8921